<a href="https://colab.research.google.com/github/AgentschapPlantentuinMeise/labelsFromiNaturalist/blob/main/makeHerbariumLabelFromiNat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install blabel

     |████████████████████████████████| 368kB 5.6MB/s 
     |████████████████████████████████| 225kB 19.9MB/s 
     |████████████████████████████████| 983kB 17.7MB/s 
     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 1.9MB 35.2MB/s 
     |████████████████████████████████| 51kB 6.8MB/s 
  Created wheel for pystrich: filename=pyStrich-0.8-cp36-none-any.whl size=1107398 sha256=4b14394d6ba3fdd429a7df6564075a57da44e5a5aee61dedf78d3c9f03f4c325
  Stored in directory: /root/.cache/pip/wheels/e7/be/0c/8fe0bd5d5163e625d2904f676b7fd6456f63f5907115a244a7
  Created wheel for cairocffi: filename=cairocffi-1.2.0-cp36-none-any.whl size=89552 sha256=51f4cb673d28bfe6753300c7d1aeb6567eb26119c482ee032fddf9b5e17aad67
  Stored in directory: /root/.cache/pip/wheels/40/76/48/f1effadceea83b32e7d957dd0f92db4db8b537d7b72b4ef374
Successfully built pystrich cairocffi


In [ ]:
import requests
#import re
#import matplotlib.pyplot as plt
#import numpy as np
import pandas as pd
#from urllib.parse import urlencode
#import json
from datetime import datetime
import time
from inspect import getmembers, isfunction
from blabel import LabelWriter
from dateutil import parser

In [ ]:
# Helpful site: https://api.inaturalist.org/v1/docs/#!/Observations/get_observations
# look for the taxon ids and place ids in the urls on the web site.

q = {
    "user_id": "qgroom",
    "year": "2018",
    "per_page": 100,
     "page": 1,
     "iconic_taxa": "Plantae",
}
url = "https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at"


#del bioblitzProjects
observations = pd.DataFrame()

num_pages = 3

In [ ]:
# get all the project from iNaturalist

page = int(1)
while True:
    r = requests.get(url, params=q)
    observations = pd.concat([observations, pd.DataFrame(r.json()["results"])], sort=False)

    if page > num_pages:
      break
    if page == 1:
        num_pages = int(r.json()["total_results"] / r.json()["per_page"])+1
        print(r.json()["total_results"])
        print(num_pages)
        print("Results:", r.json()["total_results"], ", ", num_pages, " pages total")
    print("Retrieving page ", url+" page " + str(page))
    page += 1
    q["page"] = page
    time.sleep(1)

1211
13
Results: 1211 ,  13  pages total
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 1
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 2
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 3
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 4
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 5
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 6
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order=desc&order_by=created_at page 7
Retrieving page  https://api.inaturalist.org/v1/observations?user_id=qgroom&year=2020&order

In [ ]:
tmpObservation = dict()
listOfObservations = []

for index, value in observations.iterrows():
    #print("value ",value)
    if not (value['taxon'] is None):
        for key, taxon in value['taxon'].items():
            if key == 'name':
                tmpObservation['scientificName'] = taxon
    if not (value['user'] is None):
        for key, user in value['user'].items():
            #print(key,user)
            if key == 'name':
                tmpObservation['recordedBy'] = user
            if key == 'orcid':
                tmpObservation['recordedById'] = user
    if not (value['uri'] is None):
            tmpObservation['uri'] = value['uri']
    if not (value['place_guess'] is None):
            tmpObservation['locality'] = value['place_guess']
    if not (value['location'] is None):
            tmpObservation['decimalLongitude'] = value['location'].split(',')[1]
            tmpObservation['decimalLatitude'] = value['location'].split(',')[0]
    if not (value['time_observed_at'] is None):
            #from datetime import datetime, timedelta, date
            #eventDate = datetime.strptime(value['time_observed_at'], '%Y-%m-%dT%H:%M:%S%z')
            eventDate =  parser.parse(value['time_observed_at'])
            tmpObservation['eventDate'] = eventDate.strftime('%Y-%m-%d')
            tmpObservation['eventTime'] = eventDate.strftime('%H:%M:%S%z')
    if not (value['ofvs'] is None):
            for extra in value['ofvs']:
              if extra['name'] == 'Collection #':
                tmpObservation['recordNumber'] = extra['value']
                print('Collection number ',extra['value'])
    #print(tmpObservation)
    listOfObservations.append(tmpObservation.copy())

Collection number  18091
Collection number  18032
Collection number  18052
Collection number  18058
Collection number  18039
Collection number  18017


In [ ]:
label_writer = LabelWriter("/content/herbTemplate.html",
                           default_stylesheets=("/content/style.css",))

FileNotFoundError: ignored

In [ ]:
label_writer.write_labels(listOfObservations, target='/content/qrcode_and_label.pdf')